In [1]:
import sys
sys.path.append(r'..') 
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

from model.model import MeNet
from model.Gdata import Data
from model.config import Config
from tqdm import tqdm
import matplotlib.pyplot as plt 
import numpy as np
import json

GPU Tools False


In [2]:
def load_json_file(fpath):
    with open(fpath, 'r') as f:
        data = json.load(f)
    return data

In [3]:
# AO_TPEFM
zern = [5,6,7,8,9,10,11]
# zern = [5]
# amp_range = [0.5]*len(zern)    
amp_range = [[-0.3,0.3]]*len(zern) 
amps = dict(zip(zern, amp_range))  
# bias = {5:0.4,6:0.4,7:0.1,8:0.1}
# net_architecture = 'biasEncoder'
bias = {5:0.2,6:0.2,11:0.2}
net_architecture = 'threeStreamRes2D'
# net_architecture = "resnet"

c = Config(zernike_amplitude_ranges=amps,bias_aber=bias,
           net_architecture = net_architecture,
           isMultiStream = 3,isRealTime = False,
           psf_na_detection=1.05, psf_units=(0.1,0.1,0.1), psf_n=1.33, 
           psf_lam_detection=0.920,)
# vars(c)

In [4]:
import numpy as np
# file_path = '../DataSet/Last_data/T/T1/'
file_path = '../DataSet/SimulationLast/R1_Uniform_coma7Mag_fournitial/Coma7_3'

# file_path = '../DataSet/AO_TPEFM/EE1/EE2/'

data = load_json_file(str(file_path+'/config.json'))

print('Bias:',data['bias_aber'])
print('Mode Ranges:',data['zernike_amplitude_ranges'])
print('NA:',data['psf_na_detection']) 
print('Wavelength:',data['psf_lam_detection'])
print('Refractive Index:',data['psf_n'])
print('PSF_units',data['psf_units'])

FileNotFoundError: [Errno 2] No such file or directory: '../DataSet/SimulationLast/R1_Uniform_coma7Mag_fournitial/Coma7_3/config.json'

In [ ]:
X =np.load(file_path+'/X.npy',allow_pickle=True)
# X = np.concatenate((X1,X2),axis=-1)
Y =np.load(file_path+'/Y.npy')

In [ ]:
X=X.item()
X=X['X1']

In [ ]:
# X =np.load(file_path+'X.npy')
X1 =np.load(file_path+'X1.npy')
X2 =np.load(file_path+'X2.npy')
X3 =np.load(file_path+'X3.npy')
# X4 =np.load(file_path+'X5.npy')
# X = np.concatenate((X1,X2),axis=-1)
Y =np.load(file_path+'Y.npy')

In [ ]:
from phasenet.zernike import ZernikeWavefront

# Y = data
rmse = []
gt = np.zeros(7)
zerns_noll = [5,6,7,8,9,10,11]
for i in range(Y.shape[0]):
    pre_tmp = dict(zip(zerns_noll,Y[i]))
    p_wavefront = ZernikeWavefront(pre_tmp, order='noll') 
    gt_tmp = dict(zip(zerns_noll,gt))
    gt_wavefront = ZernikeWavefront(gt_tmp, order='noll') 
    rmse.append(np.sqrt(np.nanmean((gt_wavefront.polynomial(32)-p_wavefront.polynomial(32))**2)))
rmse = np.array(rmse)
indices = np.where(rmse < 0.5)[0]
print('Wavefront RMS < 0.5:',len(indices))

In [ ]:
X1 = X1[indices,:,:,:]
X2 = X2[indices,:,:,:]
X3 = X3[indices,:,:,:]
X4 = X4[indices,:,:,:]


In [ ]:
X1 = np.concatenate((X1,X4),axis=-1)
X2 = np.concatenate((X2,X3),axis=-1)

In [ ]:
X = X[0:5000]
Y = Y[0:5000]

In [ ]:
indices = np.arange(len(X1))
np.random.shuffle(indices)

percen = 0.9
L = len(indices)
print("shuffle")
np.savez('../DataSet/data_train.npz',   
    X1_train=X1[indices[0:int(L*percen)]],X2_train=X2[indices[0:int(L*percen)]],X3_train=X3[indices[0:int(L*percen)]],
    X4_train=X4[indices[0:int(L*percen)]],Y_train=Y[indices[0:int(L*percen)]],
    X1_valid=X1[indices[int(L*percen):]],X2_valid=X2[indices[int(L*percen):]],X3_valid=X3[indices[int(L*percen):]],
    X4_valid=X4[indices[int(L*percen):]],Y_valid=Y[indices[int(L*percen):]])

In [ ]:
net_architecture = 'threeStreamRes2D'
isMultiStream = 3
indices = np.arange(len(X1))
np.random.shuffle(indices)
percen = 0.8
L = len(indices)
print("shuffle")
np.savez('../DataSet/data_train.npz',   
    X1_train=X1[indices[0:int(L*percen)]],X2_train=X2[indices[0:int(L*percen)]],X3_train=X3[indices[0:int(L*percen)]],Y_train=Y[indices[0:int(L*percen)]],
    X1_valid=X1[indices[int(L*percen):]],X2_valid=X2[indices[int(L*percen):]],X3_valid=X3[indices[int(L*percen):]],Y_valid=Y[indices[int(L*percen):]])

### Stack

In [ ]:
net_architecture = "resnet"
isMultiStream = 1

In [ ]:
# X = np.concatenate((X1,X2),axis=-1)
net_architecture = "resnet"
isMultiStream = False
indices = np.arange(len(X))
np.random.shuffle(indices)
percen = 0.8
L = len(indices)
print("shuffle")
np.savez('../DataSet/data_train.npz',   
    X1_train=X[indices[0:int(L*percen)]] ,Y_train=Y[indices[0:int(L*percen)]],
    X1_valid=X[indices[int(L*percen):]],Y_valid=Y[indices[int(L*percen):]])


### Model

In [ ]:
net_architecture = 'threeStreamRes2D'
isMultiStream = 3

In [ ]:
c = Config(zernike_amplitude_ranges=data['zernike_amplitude_ranges'],
           bias_aber=data['bias_aber'],planes = [0], net_architecture = net_architecture,
           isMultiStream = isMultiStream,isBlock = False,isRealTime = False,
           psf_na_detection=data['psf_na_detection'], psf_units=tuple(data['psf_units']), psf_n=data['psf_n'], 
           psf_lam_detection=data['psf_lam_detection'], noise_mean=None, noise_snr=None, noise_sigma=None)
#vars(c)

In [5]:

model = MeNet(config=c, name='test1', basedir='Models/0509')

c:\Users\ASUS\anaconda3\envs\tensorflow-gpu\lib\site-packages\csbdeep\models\base_model.py:149: UserWarning: output path for model already exists, files may be overwritten: E:\Project\MeNet-AO\training\Models\0509\test1
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


input_shape [(32, 32, 2), (32, 32, 2), (32, 32, 2)]
output_size: 7


In [ ]:
model.keras_model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model.keras_model, to_file='MTRes2D_fourierattention.png',show_shapes=True)

In [8]:
print(("MultiStream" if model.config.isMultiStream else "SingleStream"))

print("Net_architecture:",model.config.net_architecture)
print("Batch_size:",model.config.train_batch_size)
print("train_steps_per_epoch:",model.config.train_steps_per_epoch)
print("The number of predicted aberrations:",model.config.n_channel_out)
print("Range of predicted aberrations: ",model.config.zernike_amplitude_ranges)
print("Induced bias :",model.config.bias_aber)
print("Net_architecture:",model.config.net_architecture)
print("Net_Learning_rate:",model.config.train_learning_rate)
print("RealTime:",model.config.isRealTime)

MultiStream
Net_architecture: threeStreamRes2D
Batch_size: 32
train_steps_per_epoch: 1000
The number of predicted aberrations: 7
Range of predicted aberrations:  {5: [-0.3, 0.3], 6: [-0.3, 0.3], 7: [-0.3, 0.3], 8: [-0.3, 0.3], 9: [-0.3, 0.3], 10: [-0.3, 0.3], 11: [-0.3, 0.3]}
Induced bias : {5: 0.2, 6: 0.2, 11: 0.2}
Net_architecture: threeStreamRes2D
Net_Learning_rate: 0.0006
RealTime: False


In [11]:
model.config.train_learning_rate = 6e-3
model.config.train_steps_per_epoch = 1000
model.config.train_batch_size = 64
model.config.train_n_val = 32


In [12]:
model.train(epochs=100)

------------------------------
load train data...
------------------------------
e:\Project\MeNet-AO\training
(950, 7)
load finished
Epoch 1/100
1000/1000 [==============================] - 57s 57ms/step - loss: 7.1823e-04 - val_loss: 0.0156
Epoch 2/100
1000/1000 [==============================] - 56s 56ms/step - loss: 4.6139e-04 - val_loss: 0.0216
Epoch 3/100
1000/1000 [==============================] - 55s 55ms/step - loss: 3.5633e-04 - val_loss: 0.0106
Epoch 4/100
1000/1000 [==============================] - 55s 55ms/step - loss: 2.7975e-04 - val_loss: 0.0067
Epoch 5/100
1000/1000 [==============================] - 56s 56ms/step - loss: 2.4125e-04 - val_loss: 0.0059
Epoch 6/100
1000/1000 [==============================] - 55s 55ms/step - loss: 2.4034e-04 - val_loss: 0.0145
Epoch 7/100
1000/1000 [==============================] - 56s 56ms/step - loss: 1.8003e-04 - val_loss: 0.0089
Epoch 8/100
1000/1000 [==============================] - 56s 55ms/step - loss: 1.7252e-04 - val_loss: 0.

In [ ]:
import tensorboard